# 1. Naver Open API 접속해서 검색 하기

## (1) Naver developers 사이트 접속 (검색 API 사용 등록)

- client_id 생성
- client_secret 생성

## (2) request로 호출하기

- request = urllib.request.Request(url)
- request.add_header("X-Naver-Client-Id",client_id)
- request.add_header("X-Naver-Client-Secret",client_secret)
- response = urllib.request.urlopen(request)

In [1]:
import os
import sys
import urllib.request

# client_id = "YOUR_CLIENT_ID" # Naver에서 등록하여 받은 ID
client_id = "FoRRC_Kn5hwxBkPUu1nn" # Naver에서 등록하여 받은 ID
client_secret = "MyJjifjpnV" # Naver에서 등록하여 받은 ID

# encText = urllib.parse.quote("검색할 단어")
encText = urllib.parse.quote("축구 아시안컵")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Thu, 06 Mar 2025 16:01:06 +0900",
	"total":127912,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"u20 <b>아시안컵<\/b> 중계 4강 <b>축구<\/b> 한국 사우디 대한민국... ",
			"link":"https:\/\/blog.naver.com\/agla1\/223774522790",
			"description":"✅ afc u20 <b>아시안컵<\/b> <b>축구<\/b> 중계 보러가기 mbc sports + 채널 번호를 보았을 때 KT 올레 TV에서는 60번, SK브로드밴드의 Btv에서는 983번, LG U+TV에서는 106번으로 설정되어 있어요. 스카이라이프 사용자는 205번에서... ",
			"bloggername":"소소한 매일",
			"bloggerlink":"blog.naver.com\/agla1",
			"postdate":"20250226"
		},
		{
			"title":"U20 <b>아시안컵<\/b> 대한민국 사우디 <b>축구<\/b> 중계 경기시간 한국... ",
			"link":"https:\/\/blog.naver.com\/mazic123\/223774655239",
			"description":"U20 <b>아시안컵<\/b> 대한민국 사우디 <b>축구<\/b> 중계 경기시간 한국 사우디아라비아 4강 준결승 방송 채널 모바일 TV 실시간 하이라이트 대한민국 <b>축구<\/b> 대표팀이 U20 <b>아시안컵<\/b>에서 극적인 4강 진출을 이뤘습니다. 2025 AFC U... ",
			"bloggername":"추억만들기",
			"bloggerlink":"blog.naver.com\/mazic123",
			"postdate":"20250226"
		},
		{
			"title":"한국 사우디아라비아 <b>축구 아시안컵<\/b> 중계 보러가기",
			"link":"https:\/\/b

# 2. Naver news 크롤링 하기

- 네이버 뉴스에서 월드컵 관련 기사 크롤링 해서 파일로 저장하기

## (1) 전체 작업 설계하기

1. 검색어 지정하기

        srcText = '월드컵'

2. 네이버 뉴스 검색하기

        getNaverSearch()

        (1) url 구성하기

            - url = base + node + srcText
        
        (2) url 접속과 검색 요청하기

            - urllib.request.urlopen()

        (3) 요청 결과를 응답 json으로 받기

            - json.load()

3. 응답 데이터를 정리하여 리스트에 저장하기

        getPostData()

4. 리스트를 JSON 파일로 저장하기

        json.dumps()


In [3]:
import os
import sys
import urllib.request
import datetime
import time
import json

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" %(datetime.datetime.now(), url))
        return None


def getNaverSearch(node, srcText, page_start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(srcText), page_start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description':description, 'org_link': org_link, 'link': org_link, 'pDate':pDate})
    return

def main():
    node = 'news' # 크롤링 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)

    print('가져온 데이터 : %d 건' %(cnt))
    print('%s_naver_%s.json SAVED' %(srcText, node))

if __name__ == '__main__':
    main()

검색어를 입력하세요: 미키17
[2025-03-06 07:03:48.694796] Url Request Success
[2025-03-06 07:03:50.327851] Url Request Success
[2025-03-06 07:03:51.583634] Url Request Success
[2025-03-06 07:03:52.840852] Url Request Success
[2025-03-06 07:03:54.163789] Url Request Success
[2025-03-06 07:03:55.626871] Url Request Success
[2025-03-06 07:03:56.927845] Url Request Success
[2025-03-06 07:03:58.273090] Url Request Success
[2025-03-06 07:03:59.599865] Url Request Success
[2025-03-06 07:04:00.913816] Url Request Success
HTTP Error 400: Bad Request
[2025-03-06 07:04:01.928553] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EB%AF%B8%ED%82%A417&start=1001&display=100
전체 검색 : 19132 건
가져온 데이터 : 1000 건
미키17_naver_news.json SAVED


### Naver 검색 url

- 뉴스 : https://openapi.naver.com/v1/search/news.json
- 블로그 : https://openapi.naver.com/v1/search/blog.json
- 카페 : https://openapi.naver.com/v1/search/cafearticle.json
- 영화 : https://openapi.naver.com/v1/search/movie.json
- 쇼핑 : https://openapi.naver.com/v1/search/shop.json

### 요청 변수

- query : 검색을 원하는 문자열이며 UTF-8로 인코딩한다
- start : 검색 시작 위치로 최대 1000까지 가능 (기본값 1)
- display : 검색 결과 출력 건수를 지정 10(기본값) ~ 100(최대값)

### 응답 변수
- items : 검색 결과로 title, originallink, link, description, pubData를 포함
- title : 검색 결과 문서의 제목
- link : 검색 결과 문서를 제공하는 네이버의 하이퍼텍스트 link
- originallink : 검색 결과 문서를 제공하는 언론사의 하이퍼텍스트 link
- description : 검색 결과 문서의 내용을 요약한 정보
- pubDate : 검색 결과 문서가 네이버에 제공된 시간
